In [133]:
import pandas as pd
import requests
from datetime import date

## construct api endpoint

In [134]:
base = 'https://api.coindesk.com/v1/bpi/historical/close.json?'

In [135]:
start_date = '2011-01-01' # 2011 is first year with full year data
end_date = str(date.today())
print("today's date: " + end_date)

today's date: 2021-02-11


In [136]:
url = base + 'start=' + start_date + '&end=' + end_date
print(url)

https://api.coindesk.com/v1/bpi/historical/close.json?start=2011-01-01&end=2021-02-11


## make api call

In [137]:
df = pd.read_json(url)

## formatting

In [138]:
df.reset_index(inplace = True)
df.drop(columns = ['disclaimer','time'], inplace = True)
df.rename(columns = {'index': 'date','bpi': 'price'}, inplace = True)
df = df[df['price'].notnull()]
df['date'] = pd.to_datetime(df['date'])

In [139]:
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day

cols = ['date','year','month','day','price']
df = df[cols]

In [140]:
df.head()

,date,year,month,day,price
0,2011-01-01,2011,1,1,0.300
1,2011-01-02,2011,1,2,0.300
2,2011-01-03,2011,1,3,0.295
3,2011-01-04,2011,1,4,0.299
4,2011-01-05,2011,1,5,0.299


In [141]:
df.tail()

,date,year,month,day,price
3689,2021-02-06,2021,2,6,39269.3417
3690,2021-02-07,2021,2,7,38862.3500
3691,2021-02-08,2021,2,8,46436.0900
3692,2021-02-09,2021,2,9,46502.2933
3693,2021-02-10,2021,2,10,44855.6167


## create summary table by year
* confirm data quality: there is one price record for everyday in each year (except 2021, which is still ongoing at the time of this pull)
* observation: since 2015, every incremental year has a higher min and median price than previous years

In [142]:
# create list of metrics i want to see for bitcoin's price each year
metrics = ['count','max','min','mean','median']

# create data frame with one row for every year in the bitcoin dataset
df_summary = pd.DataFrame(df['year'].unique(), columns = ['year']) 

# calculate metric for each year
for metric in metrics:
    metric_txt = "df.groupby('year')['price']." + metric + "().to_frame('price_" + metric + "').reset_index()"
    metric_eval = eval(metric_txt)
    metric_eval.iloc[:,1] = metric_eval.iloc[:,1].apply('{:,.0f}'.format)
    df_summary = df_summary.merge(metric_eval, how = 'left', left_on = 'year', right_on = 'year')

# show summary table
df_summary

,year,price_count,price_max,price_min,price_mean,price_median
0,2011,365,30,0,6,3
1,2012,366,14,4,8,7
2,2013,365,"1,147",13,189,112
3,2014,365,951,310,527,502
4,2015,365,466,177,272,248
5,2016,366,978,359,567,582
6,2017,365,"19,343",776,"3,990","2,594"
7,2018,365,"17,136","3,214","7,529","6,896"
8,2019,365,"12,907","3,386","7,380","7,807"
9,2020,366,"28,956","4,914","11,111","9,701"


## compare yearly price performance

In [143]:
year_start_price = df[(df['month'] == 1) & (df['day'] == 1)].drop(columns = ['date','month','day'])
year_end_price = df[(df['month'] == 12) & (df['day'] == 31)].drop(columns = ['date','month','day'])

In [144]:
yearly_price_change = year_start_price.merge(year_end_price, how='inner', on='year', suffixes=['_start_of_year','_end_of_year'])
yearly_price_change['year_price_change'] = (yearly_price_change['price_end_of_year'] - yearly_price_change['price_start_of_year']) / yearly_price_change['price_start_of_year']

yearly_price_change['price_start_of_year'] = yearly_price_change['price_start_of_year'].apply('{:,.0f}'.format)
yearly_price_change['price_end_of_year'] = yearly_price_change['price_end_of_year'].apply('{:,.0f}'.format)
yearly_price_change['year_price_change'] = yearly_price_change['year_price_change'].apply('{:.0%}'.format)

In [145]:
df_summary = df_summary.merge(yearly_price_change,how='left',on='year')
df_summary

,year,price_count,price_max,price_min,price_mean,price_median,price_start_of_year,price_end_of_year,year_price_change
0,2011,365,30,0,6,3,0,5,1474%
1,2012,366,14,4,8,7,5,14,156%
2,2013,365,"1,147",13,189,112,13,757,5594%
3,2014,365,951,310,527,502,770,320,-59%
4,2015,365,466,177,272,248,314,430,37%
5,2016,366,978,359,567,582,434,968,123%
6,2017,365,"19,343",776,"3,990","2,594",998,"13,860",1289%
7,2018,365,"17,136","3,214","7,529","6,896","13,412","3,740",-72%
8,2019,365,"12,907","3,386","7,380","7,807","3,869","7,182",86%
9,2020,366,"28,956","4,914","11,111","9,701","7,188","28,956",303%
